In [1]:
import numpy as np
import random as random
import simpleguitk as simplegui
import time

In [2]:
class Card:
    global dict_col
    #display color with dictionary
    dict_col={-2:'blue',-1:'blue',0:'cyan',1:'green',2:'green',3:'green',4:'green',5:'yellow',6:'yellow',7:'yellow',8:'yellow',9:'red',10:'red',11:'red',12:'red'}
    def __init__(self, number):
        self.number = number
        
        self.color = dict_col[number]
        #open default is closed
        self.open =False
        self.position=[None,None]
    def __str__(self):
        if self.open==False:
            return "Card has Value "+str(self.number)+" and is closed."+ \
                " Its position is at "+str(self.position)
        if self.open==True:
            return "Card has Value "+str(self.number)+" and is open."+ \
                " Its position is at "+str(self.position)
        #set card to state
    def set_state(self,state):
        self.open=state
    def set_number(self,number):
        self.number=number
        self.color = dict_col[number]
    #set position    
    def set_position(self,posinput):
        self.position=posinput         
    def draw(self,canvas):
        #self.position=list(pos)
        card_size=[70,50]
        corners=[[self.position],[self.position[0]+card_size[0],self.position[1]],[self.position[0]+card_size[0],self.position[1]+card_size[1]],[self.position[0],self.position[1]+card_size[1]]]
        if self.open==False:
            canvas.draw_polygon(corners,1,'Black','Black')
        else:    
            canvas.draw_polygon(corners,1,'Black','Light Gray')
            centerb=list(self.position)
            centerb[1]+=58
            centerb[0]+=5
            canvas.draw_text(self.number,centerb,50,self.color)

Check the class functions below. Display still needs to be added. 

In [3]:
card1=Card(-2)
print(card1.color)
print(card1.open)
ar=np.zeros((2))
#number needs to be in array Crad does not work
#not sure whether Object Oriented is useful for speed, clearer for understandable
ar[0]=card1.number
ar[1]=card1.open
print(ar)
print(card1)
card1.set_state(True)
print(card1)
card1.set_state(False)
print(card1)

blue
False
[-2.  0.]
Card has Value -2 and is closed. Its position is at [None, None]
Card has Value -2 and is open. Its position is at [None, None]
Card has Value -2 and is closed. Its position is at [None, None]


Class which creates (lists) can be created from scratch or from another pile/set.

In [4]:
class Pile:
    def __init__(self,mode,take_from):
        #create new close pile, shuffles it also 
        if mode=='create_closed':
            #all closed when 
            self.open=False
            #postion in gui
            self.position=[100,10]
            #number of cards per type
            dict_num={-2:5,-1:10,0:15,1:10,2:10,3:10,4:10,5:10,6:10,7:10,8:10,9:10,10:10,11:10,12:10}
            self.list_cards=[]
            for i in range(15):
                for j in range(dict_num[i-2]):
                    cardin=Card(i-2)
                    cardin.set_position(self.position)
                    cardin.set_state(False)
                    self.list_cards.append(cardin)     
                    self.list_cards
            random.shuffle(self.list_cards)     
        #create open pile from other    
        if mode=='create_open': 
            #postion in gui
            self.position=[300,10]            
            self.list_cards=[]
            self.list_cards.append(take_from.list_cards.pop(-1))
            self.list_cards[0].set_state(True)
            self.open=True
            self.list_cards[0].set_position(self.position)            
    def __str__(self): 
        if self.open==False:
            return "Pile has length of "+str(len(self.list_cards))+" Cards and is closed."+ \
                " Its position is at "+str(self.position)
        if self.open==True:
            return "Pile has length of "+str(len(self.list_cards))+" Cards and is open."+ \
                " Its position is at "+str(self.position)
    #giving default is top most cards
    def give_card(self,position=-1):
        card=self.list_cards.pop(position)
        return card    
    def get_card(self,card):
        self.list_cards.append(card)
        #set to position and state of pile
        self.list_cards[-1].set_position(self.position)
        self.list_cards[-1].set_state(self.open)
    #recreate pile if nearly empty. paremeters, pile to recreated, pile to be used        
    def refill(self,p_open):
        #refill if there are less than 3 cards
        if len(self.list_cards)<3:
            while len(p_open.list_cards)>1:
                #card closed to bottom is taken until there is only 1 card
                #cannot use method here in line below, then the top most is taken many times
                card_a=p_open.list_cards.pop(0)
                self.get_card(card_a)
            #shuffle 
            random.shuffle(self.list_cards)   
            return p_open

Create new pile and test that it looks right. 

In [5]:
pile_closed=Pile('create_closed',False)
print(pile_closed)
for i in range(len(pile_closed.list_cards)):
    print(pile_closed.list_cards[i])  

Pile has length of 150 Cards and is closed. Its position is at [100, 10]
Card has Value 8 and is closed. Its position is at [100, 10]
Card has Value 12 and is closed. Its position is at [100, 10]
Card has Value -1 and is closed. Its position is at [100, 10]
Card has Value 10 and is closed. Its position is at [100, 10]
Card has Value 5 and is closed. Its position is at [100, 10]
Card has Value 5 and is closed. Its position is at [100, 10]
Card has Value 2 and is closed. Its position is at [100, 10]
Card has Value 7 and is closed. Its position is at [100, 10]
Card has Value 1 and is closed. Its position is at [100, 10]
Card has Value 3 and is closed. Its position is at [100, 10]
Card has Value 10 and is closed. Its position is at [100, 10]
Card has Value 6 and is closed. Its position is at [100, 10]
Card has Value 12 and is closed. Its position is at [100, 10]
Card has Value 11 and is closed. Its position is at [100, 10]
Card has Value -2 and is closed. Its position is at [100, 10]
Card 

Now test make second open pile.

In [6]:
pile_open=Pile('create_open',pile_closed)
print(pile_open)
print(len(pile_open.list_cards),len(pile_closed.list_cards))

Pile has length of 1 Cards and is open. Its position is at [300, 10]
1 149


In [7]:
for i in range(len(pile_open.list_cards)):
    print(pile_open.list_cards[i]) 

Card has Value 6 and is open. Its position is at [300, 10]


In [8]:
#testing of giving and getting cards
card_a=pile_closed.give_card()
print(card_a)
print(len(pile_closed.list_cards))
print(pile_closed.list_cards[-1])
pile_open.get_card(card_a)
print(len(pile_open.list_cards),pile_open.list_cards[-1])
#works

Card has Value 3 and is closed. Its position is at [100, 10]
148
Card has Value 4 and is closed. Its position is at [100, 10]
2 Card has Value 3 and is open. Its position is at [300, 10]


In [12]:
#emptyning closed pile to check refill method
print(len(pile_closed.list_cards),len(pile_open.list_cards))
while len(pile_closed.list_cards)>2:
    card_a=pile_closed.give_card()
    pile_open.get_card(card_a)
print(pile_open.list_cards[-1])  
print(len(pile_closed.list_cards),len(pile_open.list_cards))    
pile_closed.refill(pile_open)
print(len(pile_closed.list_cards),len(pile_open.list_cards))
print(pile_open.list_cards[-1]) 
print(pile_closed.list_cards[-1])     
#method works

149 1
Card has Value 8 and is open. Its position is at [300, 10]
2 148
149 1
Card has Value 8 and is open. Its position is at [300, 10]
Card has Value -1 and is closed. Its position is at [100, 10]


Now I create the players clas which has the cards. The cards are aranged in a 4 rows of 3 pattern. A Card Position can be empty. 

In [13]:
#input list of players
#random if random cards are opened, from which pile cards are taken
#for some purposes array better but here tried as list
class Player:
    #parameters, name mode (human, computer), take_from pile, level (number for mode, more orptions for computer then human) 
    def __init__(self,name,mode,take_from, level):
        #grid positions 
        self.positionx=[0,70,140,210,0,70,140,210,0,70,140,210]
        self.positiony=[0,0,0,0,50,50,50,50,100,100,100,100]
        #whether cards exist
        self.exist=[]
        self.list_cards=[]
        self.name=name
        #check mode computer (different levels)/human
        if mode=='computer':
            self.mode='computer'
        if mode=='human':
            self.mode='human'
        #level how random     
        self.level=level    
 
        for i in range(12):
            self.exist.append(1)
            self.list_cards.append(take_from.list_cards.pop(-1))
            self.list_cards[-1].set_state(False)
            pos=list([self.positionx[i],self.positiony[i]])
            self.list_cards[-1].set_position(pos) 
        #list for shuffling and set two to open , currently only randomly implemented
        if level<10:
            rang=list(range(12))
            random.shuffle(rang)
            self.list_cards[rang[0]].set_state(True)
            self.list_cards[rang[1]].set_state(True)
        #option for player to do that interactive to be added   
    #for furrther round in a game only recreate but rest stays the same
    def restart(self,take_from):
        self.exist=[]
        self.list_cards=[]   
        for i in range(12):
            self.exist.append(1)
            self.list_cards.append(take_from.list_cards.pop(-1))
            self.list_cards[-1].set_state(False)
            pos=list([self.positionx[i],self.positiony[i]])
            self.list_cards[-1].set_position(pos) 
        #list for shuffling and set two to open , currently only randomly implemented
        if level<10:
            rang=list(range(12))
            random.shuffle(rang)
            self.list_cards[rang[0]].set_state(True)
            self.list_cards[rang[1]].set_state(True) 
    def __str__(self):
        return "Player "+self.name+" is a "+self.mode+" in level "+str(self.level)+"."
    def give_card(self,i):
        card=self.list_cards[i]
        self.exist[i]=0
        self.list_cards[i]=None
        return card
    #is always open by design
    def get_card(self,card,i):
        self.list_cards[i]=card
        #set to right position and to open and to existing
        self.exist[i]=1
        pos=list([self.positionx[i],self.positiony[i]])
        self.list_cards[i].set_position(pos)
        self.list_cards[i].set_state(True)
    #function only for testing    
    #card, id of card in list, number, state     
    def set_value(self,i,number,state):
        #set to desired value and position
        self.exist[i]=1  
        self.list_cards[i].set_state(True)     
        self.list_cards[i].set_number(number)        
    #gets list of closed cards    
    def get_all_closed(self,silent=True):   
        closed=[]
        for j in range(12):
            if self.exist[j]==1:
                if self.list_cards[j].open==False:
                    closed.append(j)
        if silent==False:      
            print(self.name+" has "+str(len(closed))+" closed cards")            
        return closed     
    #gets list of open cards      
    def get_all_open(self,silent=True):   
        open2=[]
        for j in range(12):
            if self.exist[j]==1:
                if self.list_cards[j].open==True:
                    open2.append(j)
        if silent==False:
            print(self.name+" has "+str(len(open2))+" open cards")            
        return open2 
    #get list of existing cards
    def get_all_cards(self,silent=True):   
        existing=[]
        for j in range(12):
            if self.exist[j]==1:
                existing.append(j)
        if silent==False:
            print(self.name+" has "+str(len(existing))+" cards")            
        return existing    
    #get score at the end
    def get_score(self):   
        score=0
        for j in range(12):
            if self.exist[j]==1:
                score+=self.list_cards[j].number         
        return score 
    #check whether cards vanishes
    def check_vanish_cards(self):
        vanish=-1
        for j in range(3):
            #3 open of same number
            if self.exist[0+j]==True and self.exist[4+j]==True and self.exist[8+j]==True: 
                if self.list_cards[0+j].open==True and self.list_cards[4+j].open==True and self.list_cards[8+j].open==True and self.list_cards[0+j].number==self.list_cards[4+j].number and self.list_cards[0+j].number==self.list_cards[8+j].number:
                    vanish=j
        return vanish  
    #remove when they are marked vanish
    def vanish_cards(self,row):
        card1=self.list_cards[0+row]
        card2=self.list_cards[4+row]
        card3=self.list_cards[8+row]
        self.exist[0+row]=0
        self.exist[4+row]=0
        self.exist[8+row]=0
        self.list_cards[0+row]=None
        self.list_cards[4+row]=None
        self.list_cards[8+row]=None
        return card1, card2, card3              

Test code below.

In [16]:
#recreate piles to start from normal length
pile_closed=Pile('create_closed',False)
pile_open=Pile('create_open',pile_closed)
alpha=Player("alpha",'computer',pile_closed,0)
beta=Player("beta",'computer',pile_closed,0) 
print(alpha,beta)
closed=alpha.get_all_closed(silent=False)
open2=alpha.get_all_open(silent=False)
exist=beta.get_all_cards(silent=False)
exist2=alpha.get_all_cards(silent=False)
score=alpha.get_score()
print(score)
#checking vanishing 
check=alpha.check_vanish_cards()
print(check)
print(alpha.list_cards[0])
print(alpha.list_cards[4])
print(alpha.list_cards[8])
alpha.set_value(0,5,True)
alpha.set_value(4,5,True)
alpha.set_value(8,5,True)
print(alpha.list_cards[0])
print(alpha.list_cards[4])
print(alpha.list_cards[8])
check=alpha.check_vanish_cards()
print(check)
card1,card2,card3=alpha.vanish_cards(check)
print(alpha.list_cards[0])
print(alpha.list_cards[4])
print(alpha.list_cards[8])
print(card1,card2,card3)

Player alpha is a computer in level 0. Player beta is a computer in level 0.
alpha has 10 closed cards
alpha has 2 open cards
beta has 12 cards
alpha has 12 cards
78
-1
Card has Value 3 and is closed. Its position is at [0, 0]
Card has Value 11 and is open. Its position is at [0, 50]
Card has Value 12 and is closed. Its position is at [0, 100]
Card has Value 5 and is open. Its position is at [0, 0]
Card has Value 5 and is open. Its position is at [0, 50]
Card has Value 5 and is open. Its position is at [0, 100]
0
None
None
None
Card has Value 5 and is open. Its position is at [0, 0] Card has Value 5 and is open. Its position is at [0, 50] Card has Value 5 and is open. Its position is at [0, 100]


In [17]:
for i in range(12):
    print(i)
    print(exist[i])
    print(alpha.list_cards[i])

0
0
None
1
1
Card has Value 0 and is closed. Its position is at [70, 0]
2
2
Card has Value 5 and is closed. Its position is at [140, 0]
3
3
Card has Value 5 and is closed. Its position is at [210, 0]
4
4
None
5
5
Card has Value 6 and is open. Its position is at [70, 50]
6
6
Card has Value 8 and is closed. Its position is at [140, 50]
7
7
Card has Value 9 and is closed. Its position is at [210, 50]
8
8
None
9
9
Card has Value 2 and is closed. Its position is at [70, 100]
10
10
Card has Value 11 and is closed. Its position is at [140, 100]
11
11
Card has Value 6 and is closed. Its position is at [210, 100]


In [13]:
for i in range(len(alpha.list_cards)):
    print(f" {i} {alpha.list_cards[i]}")  

 0 None
 1 Card has Value 1 and is closed. Its position is at [70, 0]
 2 Card has Value 0 and is closed. Its position is at [140, 0]
 3 Card has Value 6 and is closed. Its position is at [210, 0]
 4 None
 5 Card has Value 2 and is open. Its position is at [70, 50]
 6 Card has Value 10 and is closed. Its position is at [140, 50]
 7 Card has Value 2 and is open. Its position is at [210, 50]
 8 None
 9 Card has Value -1 and is closed. Its position is at [70, 100]
 10 Card has Value 0 and is closed. Its position is at [140, 100]
 11 Card has Value -1 and is closed. Its position is at [210, 100]


In [18]:
#check giving and taking cards
print(f" {1} {alpha.list_cards[1]}")
card_a=alpha.give_card(1)
print(card_a)
print(f" {1} {alpha.list_cards[1]}")
print(pile_open.list_cards[-1])      
pile_open.get_card(card_a)      
print(pile_open.list_cards[-1])  

 1 Card has Value 0 and is closed. Its position is at [70, 0]
Card has Value 0 and is closed. Its position is at [70, 0]
 1 None
Card has Value 7 and is open. Its position is at [300, 10]
Card has Value 0 and is open. Its position is at [300, 10]


In [19]:
print(pile_closed.list_cards[-1])
card_b=pile_closed.give_card()
alpha.get_card(card_b,1)
print(f" {1} {alpha.list_cards[1]}") 
print(len(pile_closed.list_cards))    
print(pile_closed.list_cards[-1]) 

Card has Value 4 and is closed. Its position is at [100, 10]
 1 Card has Value 4 and is open. Its position is at [70, 0]
124
Card has Value 12 and is closed. Its position is at [100, 10]


Create all again to start from start state.

In [20]:
pile_closed=Pile('create_closed',False)
pile_open=Pile('create_open',pile_closed)
alpha=Player("alpha",'computer',pile_closed,0)
beta=Player("beta",'computer',pile_closed,0) 

Create list of players. Vary number for testing different cases.

In [21]:
#different test cases
#players=[alpha,beta,gamma,delta]
#players=[alpha,beta,gamma]
players=[alpha,beta]
#players=[alpha,beta,gamma,delta,epsilon,zeta,eta,theta]
#need better print function
print(players)
for i in range(len(players)):
    print(players[i])

[<__main__.Player object at 0x7f781b28dfa0>, <__main__.Player object at 0x7f781b28d0a0>]
Player alpha is a computer in level 0.
Player beta is a computer in level 0.


Function to determine which player starts.

In [23]:
#parameters, whether is first round of game, player_list, last_ended (which means last action of previous game)
#silent whether the result is printed on screen
def who_starts(first,players,last_ender,silent=True):
    #if first round of a game largest one starts
    if first==True:
        #first is default 
        starter=players[0]
        for i in range(len(players)):
            counter=0
            for j in range(12):
                #when card exists
                if players[i].exist[j]==1:
                    #if card open
                    if players[i].list_cards[j].open==1:
                        counter+=players[i].list_cards[j].number
            if i==0:
                lowcounter=counter
            if i>0 and counter>lowcounter:
                starter=players[i]
        if silent==False:
            print("Player "+str(starter.name)+" starts")        
        return starter 
    #otherwise the ender of previous round starts
    else:
        if silent==False:
            print("Player "+str(last_ender.name)+" starts") 
        return last_ender                    

In [25]:
#test function
a=who_starts(True,players,None,False)
b=who_starts(False,players,beta,False)

Player beta starts
Player beta starts


Function to make a turn, first computer option. Needs later different choices for levels. 

In [74]:
#players is only needed that higher level computer can consider other players card for choices
#possible add past scores for that purpose also
#silent whether results are printed or not
def turn(player,players,pile,discarded,silent=True):
    #global score(needed when several rounds) and inplay
    global score, in_play
    if player.mode=='computer':
        #in level 0 random 50% choice of action
        if player.level==0:
            r_number1=random.random()
            if r_number1<0.5:
                take_open=False
            else:    
                take_open=True
        if take_open==True:
            #by design
            discard=False
            if silent==False:
                print("take from open swaps with own")
            #use method for existing cards
            existing=player.get_all_cards()
            #select from them
            #randomly for level 0
            if player.level==0:
                selected=random.choice(existing)
            card_a=player.give_card(selected)
            card_b=pile_open.give_card()
            pile_open.get_card(card_a)
            player.get_card(card_b,selected)
            if silent==False:            
                print("player "+player.name+" has now "+str(player.list_cards[selected].number))
                print("open Pile has now on "+str(pile_open.list_cards[-1].number))            
        else:
            #in level 0 random 50% choice of sub action
            if player.level==0:
                r_number2=random.random()
                if r_number2<0.5:
                    discard=False
                else:    
                    discard=True
            if discard==True:
                if silent==False:                
                    print("take from closed discards it")
                #put card on open pile from closed
                card_a=pile_closed.give_card()
                pile_open.get_card(card_a)
                #open random closed cards using method
                closed=player.get_all_closed()
                #randomly for level 0
                if player.level==0:
                    selected=random.choice(closed)  
                player.list_cards[selected].set_state(True)
                if silent==False:                
                    print("player "+player.name+" opened "+str(player.list_cards[selected].number))
                    print("open Pile has now on top "+str(pile_open.list_cards[-1].number))                
            #next close card not discard mode (last mode)  
            else:
                if silent==False:
                    print("take from closed swaps with own")
                #choose from existing cards
                existing=player.get_all_cards()
                #randomly for level 0
                if player.level==0:
                    selected=random.choice(existing)
                card_a=player.give_card(selected)
                card_b=pile_closed.give_card()
                pile_open.get_card(card_a)
                player.get_card(card_b,selected)
                if silent==False:                
                    print("player "+player.name+" has now "+str(player.list_cards[selected].number))
                    print("open Pile has now in top "+str(pile_open.list_cards[-1].number))
    #check for vanishing cards
    van=player.check_vanish_cards()
    if van>-1:
        card1,card2,card3=player.vanish_cards(van)
        pile_open.get_card(card1)
        pile_open.get_card(card2)
        pile_open.get_card(card3)
        if silent==False:        
            print("3 "+str(card1.number)+" vanish from Player")
    #check whether there are still closed cards 
    closed=player.get_all_closed()
    if len(closed)==0:
        in_play=False 
        if silent==False:        
            print("player "+player.name+" opened all cards")
    return player, players, pile, discarded  

Playing some turns, it seems to work. 

In [75]:
pile_closed=Pile('create_closed',False)
pile_open=Pile('create_open',pile_closed)
alpha=Player("alpha",'computer',pile_closed,0)
beta=Player("beta",'computer',pile_closed,0) 
players=[alpha,beta]
starter=who_starts(True,players,None)
turn(alpha,players,pile_closed,pile_open,False)
print(len(alpha.get_all_closed()))

take from open swaps with own
player alpha has now 3
open Pile has now on 8
9


Now creating function for round. 

In [76]:
#parameters player names list, mode list, level, list, pause per turn (seconds), silent printing
#but will need later, also regarding collecting output
def skyjo_round(names,nature,levels,pause,first_round,silent=True):
    global in_play, players, pile_open, pile_closed
    #create piles
    pile_closed=Pile('create_closed',False)
    pile_open=Pile('create_open',pile_closed)
    #create players and add to list (probably needs mode were name is not recreated for later round in game)
    players=[]
    for i in range(len(names)):
        players.append(Player(names[i],nature[i],pile_closed,levels[i]))
    if silent==False:
        print("closed pile has "+str(len(pile_closed.list_cards))+" cards")        
    #drawing here?    
    #frame.set_draw_handler(draw)
    #determine start player
    if first_round==True:
        if silent==False:
            print("game starts")
        for i in range(len(players)):
            if silent==False:
                print(players[i])
        starter=who_starts(True,players,None)
    else: 
        starter=who_starts(False,players,last_ender)
        if silent==False:
            print("new round starts")
    if silent==False: 
        print("Player "+starter.name+" starts")
    #index of starter
    idx=players.index(starter)
    if pause>0:
        time.sleep(pause)
    #first turn
    turn(starter,players,pile_closed,pile_open,silent)
    if silent==False:
        print("closed pile has "+str(len(pile_closed.list_cards))+" cards")      
    #counter set to index of starter
    counter=idx
    in_play=True
    finisher=0
    while in_play==True:
        if pause>0:
            time.sleep(pause)
        #now alternating turn while in play
        counter+=1
        player=players[counter%len(players)]
        if silent==False:
            print("next turn is of "+player.name)
        player,players,pile_closed,pile_open=turn(player,players,pile_closed,pile_open,silent)
        if silent==False:
            print("closed pile has "+str(len(pile_closed.list_cards))+" cards")        
        finisher=counter%len(players)
        
    #one turn of the other players at the  end
    if in_play==False:
        for i in range(len(players)-1):
            if pause>0:
                time.sleep(pause)
            counter+=1
            player=players[counter%len(players)]
            if silent==False:
                print("next turn is of "+player.name)
            player,players,pile_closed,pile_open=turn(player,players,pile_closed,pile_open,silent)
            if silent==False:
                print("closed pile has "+str(len(pile_closed.list_cards))+" cards")
    scores=[]
    for i in range(len(players)):
        scores.append(players[i].get_score())   
    #need to consider who finished the round here
    #if not the unique lowest then score is doubled
    #works
    for i in range(len(players)):
        if i!=finisher:
            if scores[finisher]>=scores[i] and scores[finisher]>0:
                scores[finisher]*=2
    if silent==False:
        print("score of round is "+str(scores))            
    return scores, counter-idx, players

In [78]:
#test round # seems to works but less clear why it works and why the number of globals and returns are needed
#creates names and properties of player
names=['alpha','beta']
nature=['computer','computer']
levels=[0,0]
nn=int(1)
start_time=time.time()
result=np.zeros((4,nn))
for i in range(nn):
    scores,turns,players=skyjo_round(names,nature,levels,0,True,False)
    print(scores)
    print(turns)  
    result[0,i]=scores[0]
    result[1,i]=scores[1]
    result[2,i]=turns
stop_time=time.time()
print(stop_time-start_time)
print(result)

closed pile has 125 cards
game starts
Player alpha is a computer in level 0.
Player beta is a computer in level 0.
Player beta starts
take from closed discards it
player beta opened 1
open Pile has now on top 3
closed pile has 124 cards
next turn is of alpha
take from closed swaps with own
player alpha has now 0
open Pile has now in top 8
closed pile has 123 cards
next turn is of beta
take from closed swaps with own
player beta has now 7
open Pile has now in top 7
closed pile has 122 cards
next turn is of alpha
take from closed swaps with own
player alpha has now 5
open Pile has now in top 0
closed pile has 121 cards
next turn is of beta
take from open swaps with own
player beta has now 0
open Pile has now on 12
closed pile has 121 cards
next turn is of alpha
take from open swaps with own
player alpha has now 12
open Pile has now on 5
closed pile has 121 cards
next turn is of beta
take from closed discards it
player beta opened 9
open Pile has now on top 9
closed pile has 120 cards
nex

In [ ]:
Refill of closed need to be implemented and then output collection. 

Function to draw the results in the GUI  screen.  Should be disactivated for computer only.

In [ ]:
# draw function
def draw(canvas):
    global pile_open,pile_closed, players
    #display the top most card
    p_open=pile_open.list_cards[-1]
    p_closed=pile_closed.list_cards[-1]
    # test to make sure that card.draw works
    p_open.draw(canvas)
    p_closed.draw(canvas)
    for i in range(len(players)):
        for j in range(12):
            #only cards which exist are drawn:
            if players[i].exist[j]==1:
                card=players[i].list_cards[j]
                if len(players)==2:
                    drawpos=list(card.position)
                    drawpos[0]=i*290+players[i].positionx[j]
                    drawpos[1]=100+players[i].positiony[j]
                    card.set_position(drawpos)
                    card.draw(canvas)
                    canvas.draw_text(players[i].name,(100+(i%2)*290,180*(1+(i//2))+90),15,'Black')
                if abs(len(players)-3.5)==0.5:
                    drawpos=list(card.position)
                    drawpos[0]=(i%2)*290+players[i].positionx[j]
                    drawpos[1]=180*(i//2)+100+players[i].positiony[j]
                    card.set_position(drawpos)
                    card.draw(canvas)
                    canvas.draw_text(players[i].name,(100+(i%2)*290,180*(1+(i//2))+90),15,'Black')
                if abs(len(players)-5.5)==0.5:
                    drawpos=list(card.position)
                    drawpos[0]=(i%3)*290+players[i].positionx[j]
                    drawpos[1]=180*(i//3)+100+players[i].positiony[j]
                    card.set_position(drawpos)
                    card.draw(canvas)
                    canvas.draw_text(players[i].name,(100+(i%3)*290,180*(1+(i//3))+90),15,'Black')     
                if abs(len(players)-7.5)==0.5:
                    drawpos=list(card.position)
                    drawpos[0]=(i%4)*290+players[i].positionx[j]
                    drawpos[1]=180*(i//4)+100+players[i].positiony[j]
                    card.set_position(drawpos)
                    card.draw(canvas)
                    canvas.draw_text(players[i].name,(100+(i%4)*290,180*(1+(i//4))+90),15,'Black')    

Drawing works now for all allowed player numbers. 

In [ ]:
def hit_start():
    global start
    #better but still not really
    players,scores,turns=skyjo_round(names,nature,levels,1) 
    #only appears at the end

In [ ]:
#seems not work well in the notebook sometimes, not clear whether real problem
#likely ignored since notebook is not so important here
names=['alpha','beta']
nature=['computer','computer']
levels=[0,0]
if len(names)==2:
    frame = simplegui.create_frame("Skyjo", 290+280*(1), 100+3*50+30) 
if abs(len(names)-3.5)==0.5:
    frame = simplegui.create_frame("Skyjo", 290+280*(1), 100+3*50*2+60)
if abs(len(names)-5.5)==0.5:
    frame = simplegui.create_frame("Skyjo", 290+280*(2)+10, 100+3*50*2+60)    
if abs(len(names)-7.5)==0.5:
    frame = simplegui.create_frame("Skyjo", 290+280*(3)+20, 100+3*50*2+60)  
frame.set_canvas_background("White")
frame.add_button("start", hit_start, 200)    
frame.set_draw_handler(draw)
frame.start()
players,scores,turns=skyjo_round(names,nature,levels,1) 
#drawing of comuter does not work 

In [ ]:
exit()
